# Sympy codes to compute the inner expansion of Laplace and Helmholtz double-layer potentials on high aspect ratio ellipse

The expansions below are used in the manuscript

C. Carvalho, A. D. Kim, L. Lewis, and Z. Moitier, _Quadrature by Parity Asymptotic eXpansions (QPAX) for scattering by high aspect ratio particles_.

### Zoïs Moitier, Camille Carvalho (2021)
            
_Karlsruhe Institute of Technology, Germany_ 

_University of California Merced, USA_

In [1]:
import sympy as sy
from IPython.display import display

In [2]:
order = 3  # order of the expansion

In [3]:
ε, δ, h = sy.symbols("varepsilon delta h", real=True, positive=True)
s, t, X = sy.symbols("s t X", real=True)
μ_term = sy.symbols(f"mu0:{order}", cls=sy.Function)

## The Kernel $K^L(s, t; \varepsilon)$ for Laplace's double-layer potential

In [4]:
KL = -(ε) / (2 * sy.pi * (1 + ε ** 2 + (1 - ε ** 2) * sy.cos(s + t)))
display(KL)

-varepsilon/(2*pi*(varepsilon**2 + (1 - varepsilon**2)*cos(s + t) + 1))

## Expansion of the kernel $K^L(s, \pi - s + \varepsilon X; \varepsilon)$

In [5]:
KL_series = KL.subs(t, sy.pi - s + ε * X).series(x=ε, x0=0, n=order).expand()
display(KL_series)

-1/(pi*X**2*varepsilon + 4*pi*varepsilon) - X**2*varepsilon/(pi*X**4 + 8*pi*X**2 + 16*pi) - X**4*varepsilon/(12*pi*X**4 + 96*pi*X**2 + 192*pi) + O(varepsilon**3)

## Expansion of the function $\mu(\pi - s + \varepsilon X)$

In [6]:
μ_series = sum((ε * X) ** i * μ_(sy.pi - s) for i, μ_ in enumerate(μ_term)) + sy.O(
    ε ** order
)
display(μ_series)

mu0(pi - s) + X*varepsilon*mu1(pi - s) + X**2*varepsilon**2*mu2(pi - s) + O(varepsilon**3)

## Expansion of the integrand $K^L(s, \pi-s+\varepsilon X; \varepsilon) \, \mu(\pi - s + \varepsilon X)\varepsilon$

In [7]:
expr = (KL_series * μ_series * ε).expand()
display(expr)

-varepsilon*mu0(pi - s)/(pi*X**2*varepsilon + 4*pi*varepsilon) - X*varepsilon**2*mu1(pi - s)/(pi*X**2*varepsilon + 4*pi*varepsilon) - X**2*varepsilon**2*mu0(pi - s)/(pi*X**4 + 8*pi*X**2 + 16*pi) - X**2*varepsilon**3*mu2(pi - s)/(pi*X**2*varepsilon + 4*pi*varepsilon) - X**4*varepsilon**2*mu0(pi - s)/(12*pi*X**4 + 96*pi*X**2 + 192*pi) + O(varepsilon**3)

## Inner expansion $\int_{-\frac{\delta}{2\varepsilon}}^{\frac{\delta}{2\varepsilon}} K^L(s, \pi-s+\varepsilon X; \varepsilon) \, \mu(\pi - s + \varepsilon X) \varepsilon\, \mathrm{d}X$

In [8]:
int_expr = sy.O(ε ** 2)
for i in range(order):
    expr_term = expr.coeff(ε, i)
    for u_ in μ_term:
        expr_term_u = expr_term.coeff(u_(sy.pi - s), 1).factor()
        int_term = sy.integrate(expr_term_u, (X, -δ / (2 * ε), δ / (2 * ε))).expand()
        int_expr += (int_term * u_(sy.pi - s) * ε ** i).factor().expand()

display(int_expr)

-mu0(pi - s)*atan(delta/(4*varepsilon))/pi - delta**3*varepsilon*mu0(pi - s)/(12*pi*delta**2 + 192*pi*varepsilon**2) + O(varepsilon**2)

## The Kernel $K(s, t; \varepsilon) = K_1(s, t; \varepsilon)\log (4 \sin(\frac{s-t}{2})^2) + K_2(s, t; \varepsilon)$ for Helmholtz double-layer potential


In [9]:
ε, δ, h, k = sy.symbols("varepsilon delta h k", real=True, positive=True)
s, t, X = sy.symbols("s t X", real=True)
u_term = sy.symbols(f"u0:{order}", cls=sy.Function)

In [10]:
zε = (
    2
    * k
    * abs(sy.sin((s - t) / 2))
    * sy.sqrt(sy.cos((s + t) / 2) ** 2 + ε ** 2 * sy.sin((s + t) / 2) ** 2)
)
display(zε)

2*k*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*Abs(sin(s/2 - t/2))

In [11]:
K1 = -(zε * KL * sy.besselj(1, zε)) / 2
display(K1)

k*varepsilon*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*Abs(sin(s/2 - t/2))*besselj(1, 2*k*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*Abs(sin(s/2 - t/2)))/(2*pi*(varepsilon**2 + (1 - varepsilon**2)*cos(s + t) + 1))

In [12]:
log_term = sy.ln(4 * sy.sin((s - t) / 2) ** 2)
K2 = (
    -((sy.I * sy.pi * sy.hankel1(1, zε) + sy.besselj(1, zε) * log_term) * (KL * zε)) / 2
)
display(K2)

k*varepsilon*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*(log(4*sin(s/2 - t/2)**2)*besselj(1, 2*k*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*Abs(sin(s/2 - t/2))) + I*pi*hankel1(1, 2*k*sqrt(varepsilon**2*sin(s/2 + t/2)**2 + cos(s/2 + t/2)**2)*Abs(sin(s/2 - t/2))))*Abs(sin(s/2 - t/2))/(2*pi*(varepsilon**2 + (1 - varepsilon**2)*cos(s + t) + 1))

## Expansion of the kernel $K_1(s, \pi - s + \varepsilon X; \varepsilon)\log (4 \sin(\frac{s-t}{2})^2)u(s, \pi - s + \varepsilon X; \varepsilon)\varepsilon$


In [13]:
def expansion(expr):
    return expr.subs(t, sy.pi - s + ε * X).series(x=ε, x0=0, n=order).expand()


dist = (
    sy.sqrt(2)
    * sy.Abs(sy.sin((s - t) / 2))
    * sy.sqrt(1 + ε ** 2 + (1 - ε ** 2) * sy.cos(s + t))
)
z_series = expansion(k * dist).subs(sy.cos(s) * sy.sign(sy.cos(s)), sy.Abs(sy.cos(s)))

J1 = sy.besselj(1, z_series.expand().removeO())
J1_series = J1.series(x=ε, x0=0, n=order)

u_series = sum((ε * X) ** i * u_(sy.pi - s) for i, u_ in enumerate(u_term)) + sy.O(
    ε ** order
)
log_term = sy.ln(1 + ε ** 2 + (1 - ε ** 2) * sy.cos(s + t))
log_series = expansion(log_term)

expr1 = (ε * z_series * u_series * J1_series * KL_series * log_series).expand()
display(expr1)

-4*k**2*varepsilon**3*u0(pi - s)*log(varepsilon)*cos(s)**2/(pi*X**2*varepsilon + 4*pi*varepsilon) + 4*k**2*varepsilon**3*u0(pi - s)*log(2)*cos(s)**2/(2*pi*X**2*varepsilon + 8*pi*varepsilon) - 4*k**2*varepsilon**3*u0(pi - s)*log(X**2 + 4)*cos(s)**2/(2*pi*X**2*varepsilon + 8*pi*varepsilon) - 4*X*k**2*varepsilon**4*u1(pi - s)*log(varepsilon)*cos(s)**2/(pi*X**2*varepsilon + 4*pi*varepsilon) - 4*X*k**2*varepsilon**4*u0(pi - s)*log(varepsilon)*sin(s)*Abs(cos(s))*sign(cos(s))/(pi*X**2*varepsilon + 4*pi*varepsilon) + 4*X*k**2*varepsilon**4*u1(pi - s)*log(2)*cos(s)**2/(2*pi*X**2*varepsilon + 8*pi*varepsilon) - 4*X*k**2*varepsilon**4*u1(pi - s)*log(X**2 + 4)*cos(s)**2/(2*pi*X**2*varepsilon + 8*pi*varepsilon) + 4*X*k**2*varepsilon**4*u0(pi - s)*log(2)*sin(s)*Abs(cos(s))*sign(cos(s))/(2*pi*X**2*varepsilon + 8*pi*varepsilon) - 4*X*k**2*varepsilon**4*u0(pi - s)*log(X**2 + 4)*sin(s)*Abs(cos(s))*sign(cos(s))/(2*pi*X**2*varepsilon + 8*pi*varepsilon) - X**2*k**2*varepsilon**3*u0(pi - s)*log(varepsilon)*

## Expansion of the kernel $K_2(s, \pi - s + \varepsilon X; \varepsilon)u(s, \pi - s + \varepsilon X; \varepsilon)\varepsilon$
In this case we expand the Hankel function of the first kind and first order using the NIST handbook of mathematical functions [https://dlmf.nist.gov/10.7]

In [14]:
Y1approx = -2 / (sy.pi * z_series) + (2 / sy.pi) * sy.ln(z_series / 2) * J1_series
Y1_series = Y1approx.series(x=ε, x0=0, n=order)

expr2 = (
    ε
    * z_series
    * u_series
    * KL_series
    * (J1_series * log_series + sy.I * sy.pi * J1_series - sy.pi * Y1_series)
    / 2
).expand()
display(expr2)

-varepsilon*u0(pi - s)/(pi*X**2*varepsilon + 4*pi*varepsilon) - X*varepsilon**2*u1(pi - s)/(pi*X**2*varepsilon + 4*pi*varepsilon) + X*varepsilon**2*u0(pi - s)*sin(s)*Abs(cos(s))*sign(cos(s))/(2*pi*X**2*varepsilon*cos(s)**2 + 8*pi*varepsilon*cos(s)**2) - X*varepsilon**2*u0(pi - s)*sin(s)*sign(cos(s))/(2*pi*X**2*varepsilon*Abs(cos(s)) + 8*pi*varepsilon*Abs(cos(s))) + O(varepsilon**2)

## Inner expansion $\int_{-\frac{\delta}{2\varepsilon}}^{\frac{\delta}{2\varepsilon}} K_1(s, \pi-s+\varepsilon X; \varepsilon)\log (4 \sin(\frac{s-t}{2})^2) \, u(\pi - s + \varepsilon X) \varepsilon\, \mathrm{d}X$

In [15]:
int_expr1 = sy.O(ε ** 2)
for i in range(order):
    expr1_term = expr1.coeff(ε, i)
    for u_ in u_term:
        expr1_term_u = expr1_term.coeff(u_(sy.pi - s), 1).factor()
        int_term1 = sy.integrate(expr1_term_u, (X, -δ / (2 * ε), δ / (2 * ε))).expand()
        int_expr1 += (int_term1 * u_(sy.pi - s) * ε ** i).factor().expand()

display(int_expr1)

O(varepsilon**2)

## Inner expansion $\int_{-\frac{\delta}{2\varepsilon}}^{\frac{\delta}{2\varepsilon}} K_2(s, \pi-s+\varepsilon X; \varepsilon) u(\pi - s + \varepsilon X) \varepsilon\, \mathrm{d}X$


In [16]:
int_expr2 = sy.O(ε ** 2)
for i in range(order):
    expr2_term = expr2.coeff(ε, i)
    for u_ in u_term:
        expr2_term_u = expr2_term.coeff(u_(sy.pi - s), 1).factor()
        int_term2 = sy.integrate(expr2_term_u, (X, -δ / (2 * ε), δ / (2 * ε))).expand()
        int_expr2 += (int_term2 * u_(sy.pi - s) * ε ** i).factor().expand()

display(int_expr2)

-u0(pi - s)*atan(delta/(4*varepsilon))/pi + O(varepsilon**2)